In [1]:
pip install pycryptodome

     |████████████████████████████████| 13.7MB 303kB/s 


In [ ]:
from Crypto.Cipher import AES

key = b'0123456789abcdef'
IV = b'0123456789abcdef'         # Initialization vector: discussed later
mode = AES.MODE_CBC
encryptor = AES.new(key, mode, IV=IV)
text = b'jjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii'
ciphertext = encryptor.encrypt(text)

In [ ]:
ciphertext

b'\xca<w(\x8c%\xf6\x05\xe6\x8e0\tU\xe1\xa3\xd8\xacR\x1a\xbe\xa7}\x13\xf9\xe7\x14\xbd\x86\x13\xdc\xa3\x99\xd9\xc8V6\x9f\xba\x14L\xad,z~\xd8\x93\x0bt\x03\xe0\xe2\xc7\x8e;R&\x0f\x04ZM\xf0"9\x83\xce4`\x8av\x03\x0c3\x1f\xd0\xea\xe6X2&ci\x86\x8a\x9b\x05\x83\x7f~v\x81\xfb\xed\xc0_\x93\xff\xac\nO\x85\x99+\xa0\xc2K~\x06k\x19\\\xef\xa1\xb2\x9c\xdc\xc5t\xe0\xf3\xa4\x14\xc3r\xc7\xc1\x94!tY\xe2\x02\xaf\xba?\x11\xee$D\x01EK\xd1B\x8dYx\x9c\xc8Z?lo\xc867\xdf\xc1\xec;\x08\xca\xf1\x88\xba\xa6W!i>\xd3\x0e\x0f\xf6\xa0\xa1]^Y2\x9e\\\xed\x0c]c?\xe6\x0fxU\xdec'

In [ ]:
import hashlib

password = b'kitty'
key = hashlib.sha256(password).digest()

In [ ]:
key

b'gs\x1f\xf5\x817\xeb9q:\xe3\x0e\xba3\xc5L\x8c\x1dT\x18\xe0\x81B\x8c\xa8\x15\xe4\xe73\xd6Om'

In [ ]:
key = b'0123456789abcdef'
IV = b'0123456789abcdef' 
decryptor = AES.new(key, mode, IV=IV)
plain = decryptor.decrypt(ciphertext)

In [ ]:
plain

b'jjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii'

In [ ]:
import os, random, struct
from Crypto.Cipher import AES

def encrypt_file(key, in_filename, out_filename=None, chunksize=64*1024):
    """ Encrypts a file using AES (CBC mode) with the
        given key.

        key:
            The encryption key - a string that must be
            either 16, 24 or 32 bytes long. Longer keys
            are more secure.

        in_filename:
            Name of the input file

        out_filename:
            If None, '<in_filename>.enc' will be used.

        chunksize:
            Sets the size of the chunk which the function
            uses to read and encrypt the file. Larger chunk
            sizes can be faster for some files and machines.
            chunksize must be divisible by 16.
    """
    if not out_filename:
        out_filename = in_filename + '.enc'

    iv = ''.join(chr(random.randint(0, 0xFF)) for i in range(16))
    encryptor = AES.new(key, AES.MODE_CBC, iv)
    filesize = os.path.getsize(in_filename)

    with open(in_filename, 'rb') as infile:
        with open(out_filename, 'wb') as outfile:
            outfile.write(struct.pack('<Q', filesize))
            outfile.write(iv)

            while True:
                chunk = infile.read(chunksize)
                if len(chunk) == 0:
                    break
                elif len(chunk) % 16 != 0:
                    chunk += ' ' * (16 - len(chunk) % 16)

                outfile.write(encryptor.encrypt(chunk))

In [3]:
from google.colab import files
files.upload()

Saving test.txt (1).enc to test.txt (1).enc


{'test.txt (1).enc': b' \x13\xc1\x97\xa7\xda\x9dz\xe5\xc2V\xa5i\xe0\x9d\xad)O\x98\xa6\x98\xee>\xc7p\x00\x9ev\xd4\xce\x0fx\x10\xc2\xe6(A\xb8\xc6\xc4Z 7\xe1\x9e\x87\xa9\xdc\xec\x1c\x91\xb3\x96\xf6E\x9dz\xc653\xb2\xcdg\xeb'}

In [ ]:
encrypt_file(key= b'0123456789abcdef',in_filename='test.txt')

TypeError: ignored

In [2]:
from Crypto import Random
from Crypto.Cipher import AES

def pad(s):
    return s + b"\0" * (AES.block_size - len(s) % AES.block_size)

def encrypt(message, key, key_size=256):
    message = pad(message)
    iv = Random.new().read(AES.block_size)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return iv + cipher.encrypt(message)

def decrypt(ciphertext, key):
    iv = ciphertext[:AES.block_size]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = cipher.decrypt(ciphertext[AES.block_size:])
    return plaintext.rstrip(b"\0")

def encrypt_file(file_name, key):
    with open(file_name, 'rb') as fo:
        plaintext = fo.read()
    enc = encrypt(plaintext, key)
    with open(file_name + ".enc", 'wb') as fo:
        fo.write(enc)

def decrypt_file(file_name, key):
    with open(file_name, 'rb') as fo:
        ciphertext = fo.read()
    dec = decrypt(ciphertext, key)
    with open(file_name[:-4], 'wb') as fo:
        fo.write(dec)


key = b'\xbf\xc0\x85)\x10nc\x94\x02)j\xdf\xcb\xc4\x94\x9d(\x9e[EX\xc8\xd5\xbfI{\xa2$\x05(\xd5\x18'

# encrypt_file('test.txt', key)

None


In [ ]:
decrypt_file(x, key)

FileNotFoundError: ignored

In [ ]:
encrypt_file('test.zip', key)

In [4]:
decrypt_file('test.txt (1).enc',key)